In [1]:
import ee
import geemap
import geopandas as gpd
import geeft
import geersd

In [3]:
ee.Initialize()

In [4]:
gdf_roi = gpd.read_file("../../scratch/moose_mountain_FT_OPC_analysis_AOI_7year_L8.shp")
gdf_roi.set_crs(4326, inplace=True)
aoi = ee.FeatureCollection(gdf_roi.__geo_interface__).geometry()

In [5]:
dataset = geersd.Landsat8SR().filterDate("2017", "2024").filterBounds(aoi).applyScalingFactor().applyCloudMask()

In [6]:
ft = geeft.compute(dataset, "SR_B5", 2)

In [7]:
Map = geemap.Map()

Map.addLayer(ft.clip(aoi))
Map.centerObject(aoi, 10)

Map

Map(center=[49.76291503423149, -102.41828841732854], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
task = ee.batch.Export.image.toDrive(
    image=ft.clip(aoi),
    region=aoi,
    scale=30,
    folder='moose-mountain-ft',
    description="moose-mountain-ft",
    crs='EPSG:4326',
    maxPixels=1e13,
    skipEmptyTiles=True
)

task.start()

In [8]:
help(ee.batch.Export.image.toDrive)

Help on function toDrive in module ee.batch:

toDrive(image, description='myExportImageTask', folder=None, fileNamePrefix=None, dimensions=None, region=None, scale=None, crs=None, crsTransform=None, maxPixels=None, shardSize=None, fileDimensions=None, skipEmptyTiles=None, fileFormat=None, formatOptions=None, priority=None, **kwargs)
    Creates a task to export an EE Image to Drive.
    
    Args:
      image: The image to be exported.
      description: Human-readable name of the task.
      folder: The name of a unique folder in your Drive account to export
        into. Defaults to the root of the drive.
      fileNamePrefix: The Google Drive filename for the export. Defaults to
        the name of the task.
      dimensions: The dimensions of the exported image. Takes either a single
        positive integer as the maximum dimension or "WIDTHxHEIGHT" where
        WIDTH and HEIGHT are each positive integers.
      region: The lon,lat coordinates for a LinearRing or Polygon specifyi